# Experimenting with chunking classifier

In [3]:
%matplotlib inline
from __future__ import print_function
import os
from pyspark import SQLContext
from pyspark.sql import Row
import pyspark.sql.functions as sql
import pyspark.sql.types as types
#from pyspark.sql.functions import udf, length
import matplotlib.pyplot as plt
import numpy
import math
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
import pyspark.ml.feature as feature

:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [5]:
# Load Processed Parquet
sqlContext = SQLContext(sc)
notes = sqlContext.read.parquet("../data/idigbio_notes.parquet")
total_records = notes.count()
print(total_records)
# Small sample of the df
notes = notes.sample(withReplacement=False, fraction=0.0001)
notes.cache()
print(notes.count())

3230857
292


In [8]:
from lib.tokens import Tokens
tokens = Tokens()
udf_tokenize = sql.udf(tokens.tokenize, types.ArrayType(types.StringType()))
notes_w_tokens = notes.withColumn('tokens', udf_tokenize(notes['document']))
notes_w_tokens\
    .select(sql.col("tokens"))\
    .show(5, truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|tokens                                                                                                                                                                   |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[Frequent, in, loamy, sand, below, outcrops, in, small, ,, rocky, ,, ungrazed, Hesperstipa, comata-Bouteloua, gracilis, prairie, remnant, at, fork, in, rd, .]           |
|[BROOD, 32]                                                                                                                                                              |
|[[, 1°44'28, '', S, 51°27'21.3, '', W, BRASIL, :, PA, ,, Melgaço, Caxiuanã, Estação, Cient., Ferreira, Pena, Trilha, 19-22.xi.2003, YPT, AP

In [9]:
from lib.pos_tags import PosTags
pos_tags = PosTags()
udf_part_of_speech = sql.udf(pos_tags.tag, types.ArrayType(
                                    types.MapType(
                                        types.StringType(),
                                        types.StringType()
                                    )
                                )
                            )

notes_w_tokens2 = notes_w_tokens.withColumn('pos', 
                                            udf_part_of_speech(notes_w_tokens['tokens']))
notes_w_tokens2\
    .select(sql.col("pos"))\
    .show(5, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|pos                                                                                                                                                                                          

In [11]:
# Write out this small set of tagged records to build a training set by manually adding a iob tag
notes_w_tokens2.write.json("../data/chunker_pre_training.json")